# Append History Data
In this notebook, data from the `pseudo-absence generation` step is further processed to add 95 days history day for all temporal variables.

To achieve this, the NASA dataset is written to a database, then queries are made to fetch data of interest.

### Create Database

In [1]:
import pandas as pd
import sqlalchemy as db
from dateutil.relativedelta import relativedelta 
import glob
import xarray

import warnings
warnings.filterwarnings("ignore")

In [2]:
# define variables

NASA_basePath = '/mnt/disks/nasa/NASA'
# NASA_basePath = 'NASA Data'

env_profilers = [
#     'Swnet_tavg', 
#     'Lwnet_tavg', 
#     'Qle_tavg', 
#     'Qh_tavg', 
#     'Qg_tavg', 
#     'Snowf_tavg', 
#     'Rainf_tavg', 
#     'Evap_tavg', 
#     'Qs_acc', 
#     'Qsb_acc', 
#     'Qsm_acc', 
    'AvgSurfT_inst', 
    'Albedo_inst', 
#     'SWE_inst', 
#     'SnowDepth_inst', 
    'SoilMoi0_10cm_inst', 
    'SoilMoi10_40cm_inst', 
    'SoilTMP0_10cm_inst', 
    'SoilTMP10_40cm_inst', 
#     'SoilTMP40_100cm_inst', 
#     'SoilTMP100_200cm_inst', 
#     'PotEvap_tavg', 
#     'ECanop_tavg', 
    'Tveg_tavg', 
#     'ESoil_tavg', 
#     'RootMoist_inst', 
#     'CanopInt_inst', 
    'Wind_f_inst', 
    'Rainf_f_tavg', 
    'Tair_f_inst',
    'Qair_f_inst', 
    'Psurf_f_inst' 
#     'SWdown_f_tavg', 
#     'LWdown_f_tavg'
]

In [ ]:
# 'AvgSurfT_inst', 
# 'Albedo_inst', 
# 'SoilMoi0_10cm_inst', 
# 'SoilMoi10_40cm_inst', 
# 'SoilTMP0_10cm_inst', 
# 'SoilTMP10_40cm_inst', 
# 'Tveg_tavg', 
# 'Wind_f_inst', 
# 'Rainf_f_tavg', 
# 'Tair_f_inst',
# 'Qair_f_inst', 
# 'Psurf_f_inst' 

In [3]:
# geo location gridding

resx, resy = (0.25, 0.25)

lat_to_bucket_id = lambda x: int((x+90)/resy)
lon_to_bucket_id = lambda x: int((x+180)/resx)

bucket_id_to_lat = lambda x: (x*resy) - 90
bucket_id_to_lon = lambda x: (x*resx) - 180

In [4]:
# date arithmetic

def add_days(current_index, days):
    return (pd.to_datetime(current_index[0]) + relativedelta(days=days), current_index[1], current_index[2])

In [5]:
# setting up database and table

table_name = "nasa_noah_data"
engine = db.create_engine('sqlite:///NASA_GLDAS_NOAH025_3H.db')
connection = engine.connect()
metadata = db.MetaData()
# nasa_noah_data = db.Table(table_name, metadata, autoload=True, autoload_with=engine)

In [ ]:
# writing all NASA data to database 

db_start_date = pd.to_datetime("2000-01-01")
db_end_date = pd.to_datetime("2021-12-31")

current_date = db_start_date
j = 0
while current_date <= db_end_date:
    if current_date.is_year_start:
        print(current_date)
    year, month, day = list(map(int, str(current_date.date()).split('-')))
    base_name = f"{NASA_basePath}/GLDAS_NOAH025_3H.A{year}{str(month).zfill(2)}"
    files_pattern = f"{base_name}{str(day).zfill(2) }*.nc4"
    try:
        data = xarray.open_mfdataset(files_pattern, parallel=True)
        data = data.mean(dim="time", skipna=True)
        data = data[env_profilers].to_dataframe().dropna(axis=0, how='all').reset_index()
        data['lat_bucket_id'] = data['lat'].apply(lat_to_bucket_id)
        data['lon_bucket_id'] = data['lon'].apply(lon_to_bucket_id)
        data['year'] = year
        data['month'] = month
        data['day']  = day
        data["date"] = pd.to_datetime(data[['month', 'day', 'year']])
        data.index += j
        data.to_sql(table_name, engine, if_exists='append')
        j = data.index[-1] + 1
    except:
        print(f"Cannot read {current_date} data")
    current_date += relativedelta(days=1)

2000-01-01 00:00:00
2002-01-01 00:00:00
2003-01-01 00:00:00
2004-01-01 00:00:00
2005-01-01 00:00:00
2006-01-01 00:00:00
2007-01-01 00:00:00


### Append History 

In [14]:
temporal_variables = [
    'Swnet_tavg',
    'Lwnet_tavg',
    'Qle_tavg',
    'Qh_tavg',
    'Qg_tavg',
    'Snowf_tavg',
    'Rainf_tavg',
    'Evap_tavg',
    'Qs_acc',
    'Qsb_acc',
    'Qsm_acc',
    'AvgSurfT_inst',
    'Albedo_inst',
    'SWE_inst',
    'SnowDepth_inst',
    'SoilMoi0_10cm_inst',
    'SoilMoi10_40cm_inst',
    'SoilMoi40_100cm_inst',
    'SoilMoi100_200cm_inst',
    'SoilTMP0_10cm_inst',
    'SoilTMP10_40cm_inst',
    'SoilTMP40_100cm_inst',
    'SoilTMP100_200cm_inst',
    'PotEvap_tavg',
    'Ecanop_tavg',
    'Tveg_tavg',
    'Esoil_tavg',
    'RootMoist_inst',
    'CanopInt_inst',
    'Wind_f_inst',
    'Rainf_f_tavg',
    'Tair_f_inst',
    'Qair_f_inst',
    'Psurf_f_inst',
    'Swdown_f_tavg',
    'LWdown_f_tavg'
]
temporal_variables = env_profilers

In [22]:
csv_filepath = 'train_val_random_v0.csv'
data = pd.read_csv(csv_filepath)

In [23]:
# February has <= 28 days
data.loc[((data['month']==2) & (data['day'] > 28)), 'day'] = 28
data["date"] = pd.to_datetime(data[['month', 'day', 'year']])
data["observation_date"] = data["date"]
data['lat_bucket_id'] = data['y'].apply(lat_to_bucket_id)
data['lon_bucket_id'] = data['x'].apply(lon_to_bucket_id)

In [24]:
stats = data[['lat_bucket_id', 'lon_bucket_id']].describe()
lat_min, lon_min = stats.loc['min']
lat_max, lon_max = stats.loc['max']
stats

,lat_bucket_id,lon_bucket_id
count,16159.000000,16159.000000
mean,435.691627,680.946284
std,11.376503,20.807735
min,401.000000,652.000000
25%,430.000000,665.000000
50%,436.000000,673.000000
75%,442.000000,695.000000
max,468.000000,736.000000


In [25]:
data = data[['date', 'lat_bucket_id', 'lon_bucket_id', 'x', 'y', 'presence', 'year', 'month', 'day', 'clay_0.5cm_mean', 'clay_5.15cm_mean', 'sand_0.5cm_mean', 'sand_5.15cm_mean', 'silt_0.5cm_mean', 'silt_5.15cm_mean', 'observation_date']]
data = data.set_index(['date', 'lat_bucket_id', 'lon_bucket_id'])
data

,,,Unnamed: 0,x,y,presence,year,month,day,SoilMoi0_10cm_inst_dekad1,SoilMoi0_10cm_inst_dekad2,SoilMoi0_10cm_inst_dekad3,clay_0.5cm_mean,clay_5.15cm_mean,sand_0.5cm_mean,sand_5.15cm_mean,silt_0.5cm_mean,silt_5.15cm_mean,observation_date
date,lat_bucket_id,lon_bucket_id,,,,,,,,,,,,,,,,,
2000-01-11,442,658,1,-15.455833,20.581944,1,2000,1,11,8.912033,8.880046,10.657040,0.134760,0.140545,0.651147,0.647941,0.214100,0.211528,2000-01-11
2000-01-29,461,676,2,-10.933333,25.383333,1,2000,1,29,10.845142,10.790489,11.725162,0.161538,0.153232,0.615415,0.621584,0.223054,0.225189,2000-01-29
2000-01-04,437,655,3,-16.045278,19.471111,1,2000,1,4,4.783819,5.027987,7.008550,0.178887,0.182475,0.554956,0.554456,0.229309,0.226969,2000-01-04
2000-01-02,435,660,4,-14.760833,18.966389,1,2000,1,2,6.515716,6.332933,6.969298,0.195740,0.199793,0.574979,0.565655,0.229144,0.234419,2000-01-02
2000-01-17,442,658,5,-15.440278,20.643056,1,2000,1,17,9.319436,9.276956,11.089324,0.134416,0.140512,0.653609,0.650924,0.211982,0.208577,2000-01-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-10-08,418,687,16155,-8.125000,14.625000,0,2014,10,8,12.211150,10.220875,7.873386,0.154982,0.164436,0.665226,0.657830,0.179781,0.177740,2014-10-08
2014-10-05,438,684,16156,-8.875000,19.625000,0,2014,10,5,4.424037,3.863200,3.372636,0.230420,0.228502,0.605109,0.604785,0.164489,0.166697,2014-10-05
2014-10-04,455,685,16157,-8.625000,23.875000,0,2014,10,4,8.566788,7.910987,7.487170,0.285872,0.279326,0.496537,0.500304,0.217598,0.220379,2014-10-04


In [26]:
unique_years = data['year'].unique()

chunks = []
for year in unique_years:
    start_date = str((relativedelta(days=-95) + pd.to_datetime(f"{year}-01-01")).date())
    end_date = str((relativedelta(days=365) + pd.to_datetime(f"{year}-01-01")).date())
    print(f"Year -> From: {start_date}, To: {end_date}")
    query = db.select([nasa_noah_data]).where(db.and_(
        nasa_noah_data.columns.date >= start_date, 
        nasa_noah_data.columns.date <= end_date,
        nasa_noah_data.columns.lat_bucket_id >= 401,
        nasa_noah_data.columns.lat_bucket_id <= 468,
        nasa_noah_data.columns.lon_bucket_id >= 653,
        nasa_noah_data.columns.lon_bucket_id <= 736,
    ))
    query_result = pd.read_sql_query(query, engine).set_index(['date', 'lat_bucket_id', 'lon_bucket_id'])
    subset = data[data['year'] == year]
    for days in range(0, 96):
        indices = subset['observation_date'].index.map(lambda row: add_days(row, days=-days))
        subset_day_x = query_result.reindex(indices)
        for variable in temporal_variables:
            subset[f"{variable}_{days}"] = list(subset_day_x[variable])
    chunks.append(subset)

Year -> From: 1999-09-28, To: 2000-12-31
Year -> From: 2000-09-28, To: 2002-01-01
Year -> From: 2001-09-28, To: 2003-01-01
Year -> From: 2002-09-28, To: 2004-01-01
Year -> From: 2003-09-28, To: 2004-12-31
Year -> From: 2004-09-28, To: 2006-01-01
Year -> From: 2005-09-28, To: 2007-01-01
Year -> From: 2006-09-28, To: 2008-01-01
Year -> From: 2007-09-28, To: 2008-12-31
Year -> From: 2008-09-28, To: 2010-01-01
Year -> From: 2009-09-28, To: 2011-01-01
Year -> From: 2010-09-28, To: 2012-01-01
Year -> From: 2011-09-28, To: 2012-12-31
Year -> From: 2012-09-28, To: 2014-01-01
Year -> From: 2013-09-28, To: 2015-01-01


In [27]:
full_data = pd.concat(chunks).reset_index()
full_data.to_csv("train_val_random_v0_full.csv")

In [28]:
full_data

,date,lat_bucket_id,lon_bucket_id,Unnamed: 0,x,y,presence,year,month,day,...,SoilMoi0_10cm_inst_95,SoilMoi10_40cm_inst_95,SoilTMP0_10cm_inst_95,SoilTMP10_40cm_inst_95,Tveg_tavg_95,Wind_f_inst_95,Rainf_f_tavg_95,Tair_f_inst_95,Qair_f_inst_95,Psurf_f_inst_95
0,2000-01-11,442,658,1,-15.455833,20.581944,1,2000,1,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-29,461,676,2,-10.933333,25.383333,1,2000,1,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-04,437,655,3,-16.045278,19.471111,1,2000,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-02,435,660,4,-14.760833,18.966389,1,2000,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-17,442,658,5,-15.440278,20.643056,1,2000,1,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16154,2014-10-08,418,687,16155,-8.125000,14.625000,0,2014,10,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16155,2014-10-05,438,684,16156,-8.875000,19.625000,0,2014,10,5,...,3.566000,32.128624,311.021545,309.571167,0.0000,5.463826,0.000000,309.889679,0.005185,96857.296875
16156,2014-10-04,455,685,16157,-8.625000,23.875000,0,2014,10,4,...,6.613125,55.173374,312.428162,308.870850,0.0000,3.955625,0.000000,312.195740,0.002759,97349.679688
16157,2014-10-26,408,701,16158,-4.625000,12.125000,0,2014,10,26,...,22.513374,68.310120,302.585297,303.709167,60.9725,2.905102,0.000004,300.523132,0.015589,96744.765625


In [115]:
len(l[l['lat_bucket_id'] < 308])/len(l)

0.30720800235363344